In [ ]:
import torch
import numpy as np
from numpy.linalg import norm
from numpy.linalg import inv
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

from random import seed
from random import random

import itertools

In [ ]:
np.random.seed(42)

w1 = np.random.rand()
w3 = np.random.rand()
w2 = np.random.rand()
w4 = np.random.rand()

print(np.array([[w1,w2],[w3,w4]]))

In [ ]:
# This funtion generates data 
np.random.seed(42)
def data_generator(n):
    np.random.seed(42)
    As = []    
    x = np.random.normal(0,1,size = (2,n)) # n is the size of sample
    noise = np.random.normal(0,0.04**2) 
    
    for condition_number in [1,0.1,0.01,0.0001]: # create a list of As with different conditional number
        A = np.matrix([[1, 1], [1, 1+condition_number]])
        As.append(A)
        
    y1 = np.dot(As[0],x) + 0.01*noise #compute y with different A
    y01 = np.dot(As[1],x) + 0.01*noise
    y001 = np.dot(As[2],x) + 0.01*noise
    y0001 = np.dot(As[3],x) + 0.01*noise
    
    x = torch.Tensor(x).float() #convert all ndarrays to tensors
    y1 = torch.Tensor(y1).float()
    y01 = torch.Tensor(y01).float()
    y001 = torch.Tensor(y001).float()
    y0001 = torch.Tensor(y0001).float()
#     print(x.size())
#     print(y1.size())
    
    return x, y1, y01, y001, y0001    

In [ ]:
def Relu(Z):
    return np.maximum(0,Z)

def dRelu(x):
    return np.where(x > 0, 1.0, 0.0)

In [ ]:
class net:
    def __init__(self, x, y):
        self.X=x
        self.Y=y
        self.Yh=np.zeros((1,self.Y.shape[1]))
        self.L=2
        self.dims = [2, 4, 2]
        self.weights = {}
        self.ch = {} # a cache variable
        self.loss = []
        self.lr=0.03
        self.size = self.Y.shape[1]
    def nInit(self):    
        np.random.seed(1)
        self.weights['W'] = np.array([[w1,w2],[w3,w4]]) 
        return

    def forward(self):    
        Z1 = self.weights['W'].dot(self.X)
        self.Yh=Z1
        loss=self.calculate_loss(Z1)
        return self.Yh, loss
    
    def calculate_loss(self,Yh):
        loss = (torch.norm(self.Y-torch.from_numpy(Yh)))**2 * (1./self.size) #norm
        return loss
    
    def backward(self):

        dLoss_Yh = -2*(self.Y - torch.from_numpy(self.Yh))* (1./self.size)
        dLoss_W = np.dot(dLoss_Yh,self.X.T) 

        self.weights["W"] = self.weights["W"] - self.lr * dLoss_W


    def run(self,X, Y, epochs):
        self.loss = []
        np.random.seed(42)                         
        self.nInit()
        for i in range(0, epochs):
            Yh, loss=self.forward()
            self.backward()
            if i % 1 == 0:
#                 print ("Cost after iteration %i: %f",(i, loss))
                self.loss.append(loss)
        print("-"*50)
        print("W",self.weights["W"],"\n")
#         print("Y",Y)
#         print("Yh",Yh)
        return self.loss
        

In [ ]:
def inverse_main(n,epochs):
    np.random.seed(42)
    
    train_x_all, train_y_1,train_y_01,train_y_001,train_y_0001= data_generator(n) # create the data 
    
    nn = net(train_y_1, train_x_all)
    list_1 = nn.run(train_y_1, train_x_all,epochs)
    nn = net(train_y_01, train_x_all)
    list_01 = nn.run(train_y_01, train_x_all,epochs)
    nn = net(train_y_001, train_x_all)
    list_001 = nn.run(train_y_001, train_x_all,epochs)
    nn = net(train_y_0001, train_x_all)
    list_0001 = nn.run(train_y_0001, train_x_all,epochs)     
    
    # plot 3000 epochs and the losses
    epochs = 3000
    plt.suptitle('Simple Inverse_net - python')
    plt.plot(range(epochs),list_1)
    plt.plot(range(epochs),list_01)
    plt.plot(range(epochs),list_001)
    plt.plot(range(epochs),list_0001)
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0,14))
    plt.xlabel('Normalised Number of Epochs')
    plt.ylabel('Mean Squared Error')
    plt.legend(['1', '0.1', '0.01', '0.0001'], loc='upper right')
    plt.show()

In [ ]:
inverse_main(10000,3000)

In [ ]:
def tick_inverse_main(n,epochs):
    np.random.seed(42)
    
    train_x_all, train_y_1,train_y_01,train_y_001,train_y_0001= data_generator(n) # create the data 
    
    for index,Y in enumerate([train_y_1,train_y_01,train_y_001,train_y_0001]):
        globals()['x_tik_'+str(index)] = np.matmul(np.matmul(inv(np.matmul(As[index].T,As[index])
                                                                 +0.04**2*np.eye(2)),As[index].T),Y)
    
    nn = net(train_y_1,x_tik_0)
    list_1 = nn.run(train_y_1,x_tik_0,epochs)
    
    nn = net(train_y_01, x_tik_1)
    list_01 = nn.run(train_y_01, x_tik_1,epochs)
    
    nn = net(train_y_001, x_tik_2)
    list_001 = nn.run(train_y_001, x_tik_2,epochs)
    #print(list_001)
    
    nn = net(train_y_0001, x_tik_3)
    list_0001 = nn.run(train_y_0001, x_tik_3,epochs)  

    # plot 3000 epochs and the losses
    epochs = 3000
    plt.suptitle('Inverse Problem(tikhonov) -simple network - python')
    plt.plot(range(epochs),list_1)
    plt.plot(range(epochs),list_01)
    plt.plot(range(epochs),list_001)
    plt.plot(range(epochs),list_0001)
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0,14))
    plt.xlabel('Number of Epochs')
    plt.ylabel('Normalised Mean Squared Error')
    plt.legend(['1', '0.1', '0.01', '0.0001'], loc='upper right')
    plt.show()

In [ ]:
As = []
for condition_number in [1,0.1,0.01,0.0001]: # create a list of As with different conditional number
        A = np.matrix([[1, 1], [1, 1+condition_number]]) 
        As.append(A)
        
for index,A in enumerate(As):
        globals()['tick_approx_'+str(index)] = np.matmul(inv(np.matmul(As[index].T,As[index])+0.04**2*np.eye(2)),As[index].T)

In [ ]:
print("tick_approx_1",tick_approx_0,"\n")
print("tick_approx_0.1",tick_approx_1,"\n")
print("tick_approx_0.01",tick_approx_2,"\n")
print("tick_approx_0.0001",tick_approx_3,"\n")

In [ ]:
tick_inverse_main(10000,3000)